# Cyclistic Case Study

In [ ]:
# Load packages
import pandas as pd
import numpy as np
import glob
import datetime as dt
from haversine import haversine, Unit
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Get a list of all CSV files in the directory
csv_files = glob.glob("*.csv")

# Create an empty dataframe to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file
for csv_file in csv_files:
    # Read the CSV file into a dataframe
    df = pd.read_csv(csv_file)

    # Append the dataframe to the combined dataframe
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Print the combined dataframe
print(combined_df)

In [ ]:
combined_df.head()

In [ ]:
combined_df.info()

## Data clean
1. Remove duplicates
2. Data type: datetime
3. Missing value
4. Error: negative duration / distance
5. Outliers: extremely small or large

In [ ]:
# ride_id is the primary key. 
# Each row represents a single ride
# Remove any duplicative rides
rides = combined_df.drop_duplicates()

In [ ]:
# convert to datetime
# using mixed format: '%Y-%m-%d %H:%M:%S.%f' '%Y-%m-%d %H:%M:%S'
rides['started_at'] = pd.to_datetime(rides['started_at'],format='mixed') 
rides['ended_at'] = pd.to_datetime(rides['ended_at'],format='mixed') 

In [ ]:
# Check columns with missing value
col_with_na = rides.isna().any()
col_with_na 
# Both start and end station have missing value 
# However the start lat and lng don't. It can provide information of the start point.
# We can focus on clean the end lat and lng.

In [ ]:
end_lat_lng = rides[['end_lat','end_lng']]
end_lat_lng[end_lat_lng.isna().any(axis=1)] 
# 7441 records. 0.1 percent of the whole dataset
# It is ok to remove the missing data. 7441/5854544 

In [ ]:
# Remove na
rides = rides.dropna(subset=['end_lat','end_lng'])

In [ ]:
# Calculate duration
rides['duration_m'] = round((rides['ended_at']-rides['started_at']).dt.total_seconds()/60,2)

In [ ]:
# check how many durations are below 0 
sum(rides['duration_m']<=0)

In [ ]:
# duration should be postive
rides = rides[rides['duration_m'] > 0]

In [ ]:
# get month of the date
rides['month'] = rides['started_at'].dt.month 

In [ ]:
# get weekday of the date
rides['weekday'] = rides['started_at'].dt.day_name()
rides.head()

In [ ]:
# distance isn't the key to identify how far they ride the bike.
# because distance can be 0. Users can return the bike to the original station.
# It helps identfiy outliers.

In [ ]:
def calculate_distance(rides):
    coords_1 = (rides['start_lat'],rides['start_lng'])
    coords_2 = (rides['end_lat'],rides['end_lng'])
    return haversine(coords_1, coords_2, unit=Unit.MILES)

In [ ]:
rides['distance'] = rides.apply(calculate_distance, axis=1)

In [ ]:
# Outliers
# dataset shape 
rides.shape

In [ ]:
# max of duration is over 24 hours. 
# the potential reasons: 1. system error 2. user forgot to return the bike 3. user couldn't find a place to return.
rides['duration_m'].agg(['min','mean','max'])

In [ ]:
rides['duration_m'].describe()

In [ ]:
# calculate IQR to identify outliers
du_q1 = rides['duration_m'].describe().iloc[4]
du_q3 = rides['duration_m'].describe().iloc[6]

In [ ]:
iqr_du = du_q3 - du_q1
upper_du = du_q3+1.5*iqr_du
lower_du = du_q1-1.5*iqr_du
print(upper_du)
print(lower_du)

In [ ]:
# check under each criteria, how much data will be identified as outliers
# 120min: 35370/5846014 0.6%
# 60min: 144319/5846014 2%
# upper_du: 435825/5846014 7%

In [ ]:
rd_iqr = rides[rides['duration_m'] <= upper_du]

In [ ]:
sns.histplot(data=rd_iqr, x="duration_m",binwidth=3)

In [ ]:
rd_60 = rides[rides['duration_m'] <= 60]

In [ ]:
sns.histplot(data=rd_60, x="duration_m",binwidth=3)

In [ ]:
rd_120 = rides[rides['duration_m'] <= 120]

In [ ]:
sns.histplot(data=rd_120, x="duration_m",binwidth=3)

In [ ]:
# max of distance is over 3000 miles, which is not reasonable
rides['distance'].agg(['min','mean','max'])

In [ ]:
rides['distance'].describe()

In [ ]:
# IQR
di_q1 = rides['distance'].describe().iloc[4]
di_q3 = rides['distance'].describe().iloc[6]

In [ ]:
iqr_di = di_q3 - di_q1
upper_di = di_q3+1.5*iqr_di
lower_di = di_q1-1.5*iqr_di
print(upper_di)
print(lower_di)

In [ ]:
# check minutes per mile to identify reasonable data range
# expert biker rides 2 minutes per mile 
rides['min/mile'] = rides['duration_m']/rides['distance']

In [ ]:
di_index = rides[(rides['distance'] > upper_di) & (rides['min/mile']<2)].index #431

In [ ]:
# remove outlier based on reasonable distance 
rides = rides.drop(di_index)

In [ ]:
# remove outlier based on duration
# kept duration within 120 mins to get more insights 
rides_df = rides[rides['duration_m'] <= 120]
rides_df

In [ ]:
# count how many rides in each user group
user_group = rides_df['member_casual'].value_counts().reset_index()
user_group

In [ ]:
# set the visualization style and color panel
sns.set_style("whitegrid")
colors = sns.color_palette("pastel")[0:2]

In [ ]:
# plot the number of rides in each user group
plt.pie(user_group['count'], labels=user_group['member_casual'], colors=colors, autopct='%.0f%%') 
  
plt.show() 

## Rideable_type

In [ ]:
# count how many rides in each rideable type and user group
ride_type = rides_df.groupby(['member_casual','rideable_type'])['ride_id'].count().reset_index()

In [ ]:
ride_type['total'] = ride_type.groupby('member_casual')['ride_id'].transform(lambda x: x.sum())
ride_type['percent_of_total'] = ride_type['ride_id']/ride_type['total']*100
ride_type

In [ ]:
sns.barplot(ride_type, x="rideable_type", y="ride_id", hue="member_casual",hue_order=['member','casual'],palette=colors)

plt.xlabel("Rideable Type")
plt.ylabel("Number of Rides")
plt.title("Rideable Type")

plt.show() 

## Duration & Weekdays & Month

In [ ]:
# duration analysis
# group by and see the average and std of each user group 
rides_df.groupby('member_casual')['duration_m'].agg(['std','mean'])

In [ ]:
ri_duration = rides_df[['ride_id','duration_m','member_casual']]

In [ ]:
# plot histgram to see the duration distribution
sns.histplot(data=rides_df,x='duration_m',hue='member_casual',binwidth=3,kde=True)
plt.xlabel("Ride Duration")
plt.ylabel("Number of Rides")
plt.title("Ride Duration")

plt.show() 

In [ ]:
# assign duration labels to get a more straightforward visualization
ri_duration['label_duration'] = np.where(ri_duration['duration_m']<=10,
                                         'less than 10 mins',
                                         np.where((ri_duration['duration_m'] <= 30) & (ri_duration['duration_m']>10),
                                                  'less than 30 mins',
                                                  np.where((ri_duration['duration_m'] <= 60) & (ri_duration['duration_m']>30),
                                                           'less than 60 mins',
                                                               np.where(ri_duration['duration_m'] >60,
                                                                    'more than 60 mins',
                                                                        'nan'
                                                               
                                                           )
                                                    
                                                      
                                                  )
                                             
                                         )
                                        )

In [ ]:
ri_duration_plot = ri_duration.groupby(['member_casual','label_duration'])['ride_id'].count().reset_index()
ri_duration_plot

In [ ]:
sns.barplot(data=ri_duration_plot,x='label_duration',y='ride_id',hue='member_casual',hue_order=['member','casual'],palette=colors)
plt.xlabel("Ride Duration")
plt.ylabel("Number of Rides")
plt.title("Ride Duration")

plt.show() 

In [ ]:
# casual rider over 30 mins
c_over30 = (238152+91500)/2096237*100
m_over30 =	(188021+17447)/3713976*100
print(c_over30)
print(m_over30)

In [ ]:
# month analysis
ride_month = rides_df.groupby(['member_casual','month'])['ride_id'].count().reset_index()
ride_month['shift_count'] = ride_month['ride_id'].shift(periods=1)
ride_month['percentage_of_difference'] = (ride_month['ride_id']-ride_month['shift_count'])/ride_month['shift_count']*100
ride_month

In [ ]:
sns.barplot(data=ride_month,x='month',y='ride_id',hue='member_casual',hue_order=['member','casual'],palette=colors)
plt.xlabel("Month")
plt.ylabel("Number of Rides")
plt.title("Month Rides")
plt.show()

In [ ]:
# weekday analysis
ride_weekday = rides_df.groupby(['member_casual','weekday'])['ride_id'].count().reset_index()

# convert weekday names to numbers using the mapto sort the table
weekday_map = {    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

ride_weekday['weekday_num'] = ride_weekday['weekday'].map(weekday_map)
ride_weekday = ride_weekday.sort_values(by='weekday_num')
ride_weekday

In [ ]:
# avg weekdays vs. weekends
c_weekend = (422147+359003)/2
c_weekday = (2096237-422147-359003)/5
m_weekend = (471895+420516)/2
m_weekday = (3713976-471895-420516)/5 

print((c_weekend-c_weekday)/c_weekday*100)
print((m_weekend-m_weekday)/m_weekday*100)

In [ ]:
sns.barplot(data=ride_weekday,x='weekday',y='ride_id',hue='member_casual',hue_order=['member','casual'],palette=colors)
plt.xlabel("Weekday")
plt.ylabel("Number of Rides")
plt.title("Weekday Rides")
plt.show()

## Station analysis

In [ ]:
member_df = rides_df[rides_df['member_casual']== 'member']
casual_df = rides_df[rides_df['member_casual']== 'casual']

In [ ]:
print(sum(member_df['distance']==0)/3713976)
print(sum(casual_df['distance']==0)/2096237)

In [ ]:
print(casual_df['start_station_name'].nunique())
print(member_df['start_station_name'].nunique())

In [ ]:
casual_df.groupby(['start_station_name'])['ride_id'].count().sort_values(ascending=False).head(5)

In [ ]:
member_df.groupby(['start_station_name'])['ride_id'].count().sort_values(ascending=False).head(5)